In [1]:
!pip install opencv-python numpy fitz PyMuPDF deskew scikit-image

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.9/96.9 kB 2.1 MB/s eta 0:00:00 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 6.9 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 3.1 MB/s eta 0:00:00m eta 0:00:010:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.4/95.4 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.5/421.5 kB 3.7 MB/s eta 0:00:00m eta 0:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 528.1/528.1 kB 4.1 MB/s eta 0:00:00 MB/s eta 0:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.2/152.2 kB 2.9 MB/s eta 0:00:004.7 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 2.2 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.0 MB/s eta 0:00:00


In [62]:
#done

import subprocess
import os

pdf = "Buendia - Instruccion"
# Define the paths and parameters
pdf_path = f"/home/sharvesh/Documents/Others/Human_AI/Test_sources/Test sources/{pdf}.pdf"
output_dir = f"/home/sharvesh/Documents/Others/Human_AI/Model/data/processed_book/{pdf}/pages"
trained_model = "/home/sharvesh/Documents/Others/Human_AI/Model/CRAFT/weights/craft_mlt_25k.pth"
refiner_model = "/home/sharvesh/Documents/Others/Human_AI/Model/CRAFT/weights/craft_refiner_CTW1500.pth"
text_threshold = 0.9
test_folder = output_dir  # Images folder created by process_main_utils.py
result_folder = f"/home/sharvesh/Documents/Others/Human_AI/Model/result/"
line_segments_folder = f"/home/sharvesh/Documents/Others/Human_AI/Model/data/{pdf}/line_segments"
padding = 50  #fixed(50)
min_width = 50
threshold = 0.7
margin = 0

# New parameters for process_main_utils.py
dpi = 300
remove_borders = True
noise_removal_area_threshold = 50
intensity_threshold = 128

# Ensure output directories exist
os.makedirs(output_dir, exist_ok=True)
os.makedirs(result_folder, exist_ok=True)
os.makedirs(line_segments_folder, exist_ok=True)

# Check if the output directory is empty
if not os.listdir(output_dir):
    print("Output directory is empty. Performing Step 1: Processing PDF into individual images.")
    # Step 1: Process the PDF into individual images
    subprocess.run([
        "python3", "process_main_utils.py",
        pdf_path, output_dir,
        "--dpi", str(dpi),
        "--remove_borders",
        "--noise_removal_area_threshold", str(noise_removal_area_threshold),
        "--intensity_threshold", str(intensity_threshold)
    ])
else:
    print("Output directory is not empty. Skipping Step 1.")

# Step 2: Run the text detection model on the processed images
print("Performing Step 2: Running the text detection model.")
subprocess.run([
    "python3", "/home/sharvesh/Documents/Others/Human_AI/Model/CRAFT/test.py",
    "--trained_model", trained_model,
    "--text_threshold", str(text_threshold),
    "--test_folder", output_dir,
    "--refine",
    "--refiner_model", refiner_model
])

# Step 3: Run the contour line splitter
print("Performing Step 3: Running the contour line splitter.")
subprocess.run([
    "python3", "contour_line_splitter.py",
    output_dir,  # Image directory from the first process
    result_folder,  # Contour directory from the second process
    line_segments_folder,  # Output directory for line segments
    "--padding", str(padding),
    "--min_width", str(min_width),
    "--threshold", str(threshold),
    "--margin", str(margin)
])

print("Processing completed successfully.")


Output directory is not empty. Skipping Step 1.
Performing Step 2: Running the text detection model.


/home/sharvesh/.local/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/sharvesh/.local/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/home/sharvesh/Documents/Others/Human_AI/utsav/CRAFT/test.py:128: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only`

Loading weights from checkpoint (/home/sharvesh/Documents/Others/Human_AI/Model/CRAFT/weights/craft_mlt_25k.pth)
Loading weights of refiner from checkpoint (/home/sharvesh/Documents/Others/Human_AI/Model/CRAFT/weights/craft_refiner_CTW1500.pth)
elapsed time : 52.481603384017944socuments/Others/Human_AI/Model/data/processed_book/Buendia - Instruccion/pages/10.jpg
Performing Step 3: Running the contour line splitter.
in process dir func
Processing /home/sharvesh/Documents/Others/Human_AI/Model/data/processed_book/Buendia - Instruccion/pages/1.jpg with contours /home/sharvesh/Documents/Others/Human_AI/Model/result/res_1.txt
HI
in get func
center diff: 24.0
center diff: 752.0
center diff: 487.0
center diff: 457.0
center diff: 504.0
center diff: 355.0
center diff: 664.0
center diff: 461.0
center diff: 616.0
center diff for skipped: 1286.0
center diff: 380.0
center diff: 95.0
center diff: 11.0
center diff: 62.0
center diff: 2.0
center diff: 7.0
center diff: 51.0
center diff: 28.0
center diff

In [61]:
#done

import subprocess
import os

pdf = "Constituciones sinodales Calahorra 1602"
# Define the paths and parameters
pdf_path = f"/home/sharvesh/Documents/Others/Human_AI/Test_sources/Test sources/{pdf}.pdf"
output_dir = f"/home/sharvesh/Documents/Others/Human_AI/Model/data/processed_book/{pdf}/pages"
trained_model = "/home/sharvesh/Documents/Others/Human_AI/Model/CRAFT/weights/craft_mlt_25k.pth"
refiner_model = "/home/sharvesh/Documents/Others/Human_AI/Model/CRAFT/weights/craft_refiner_CTW1500.pth"
text_threshold = 0.9
test_folder = output_dir  # Images folder created by process_main_utils.py
result_folder = f"/home/sharvesh/Documents/Others/Human_AI/Model/result/"
line_segments_folder = f"/home/sharvesh/Documents/Others/Human_AI/Model/data/{pdf}/line_segments"
padding = 50  #fixed(50)
min_width = 50
threshold = 0.7
margin = 0

# New parameters for process_main_utils.py
dpi = 300
remove_borders = True
noise_removal_area_threshold = 50
intensity_threshold = 128

# Ensure output directories exist
os.makedirs(output_dir, exist_ok=True)
os.makedirs(result_folder, exist_ok=True)
os.makedirs(line_segments_folder, exist_ok=True)

# Check if the output directory is empty
if not os.listdir(output_dir):
    print("Output directory is empty. Performing Step 1: Processing PDF into individual images.")
    # Step 1: Process the PDF into individual images
    subprocess.run([
        "python3", "process_main_utils.py",
        pdf_path, output_dir,
        "--dpi", str(dpi),
        "--remove_borders",
        "--noise_removal_area_threshold", str(noise_removal_area_threshold),
        "--intensity_threshold", str(intensity_threshold)
    ])
else:
    print("Output directory is not empty. Skipping Step 1.")

# Step 2: Run the text detection model on the processed images
print("Performing Step 2: Running the text detection model.")
subprocess.run([
    "python3", "/home/sharvesh/Documents/Others/Human_AI/Model/CRAFT/test.py",
    "--trained_model", trained_model,
    "--text_threshold", str(text_threshold),
    "--test_folder", output_dir,
    "--refine",
    "--refiner_model", refiner_model
])

# Step 3: Run the contour line splitter
print("Performing Step 3: Running the contour line splitter.")
subprocess.run([
    "python3", "contour_line_splitter.py",
    output_dir,  # Image directory from the first process
    result_folder,  # Contour directory from the second process
    line_segments_folder,  # Output directory for line segments
    "--padding", str(padding),
    "--min_width", str(min_width),
    "--threshold", str(threshold),
    "--margin", str(margin)
])

print("Processing completed successfully.")


Output directory is empty. Performing Step 1: Processing PDF into individual images.
Total pages in PDF: 6
Processed page 1/6
Processed page 2/6
Processed page 3/6
Processed page 4/6
Processed page 5/6
Processed page 6/6
Processing completed!
Performing Step 2: Running the text detection model.


/home/sharvesh/.local/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/sharvesh/.local/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/home/sharvesh/Documents/Others/Human_AI/utsav/CRAFT/test.py:128: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only`

Loading weights from checkpoint (/home/sharvesh/Documents/Others/Human_AI/Model/CRAFT/weights/craft_mlt_25k.pth)
Loading weights of refiner from checkpoint (/home/sharvesh/Documents/Others/Human_AI/Model/CRAFT/weights/craft_refiner_CTW1500.pth)
elapsed time : 28.764047145843506suments/Others/Human_AI/Model/data/processed_book/Constituciones sinodales Calahorra 1602/pages/1.jpg
Performing Step 3: Running the contour line splitter.
in process dir func
Processing /home/sharvesh/Documents/Others/Human_AI/Model/data/processed_book/Constituciones sinodales Calahorra 1602/pages/4.jpg with contours /home/sharvesh/Documents/Others/Human_AI/Model/result/res_4.txt
HI
in get func
center diff: 10.0
center diff: 39.0
center diff: 32.0
center diff: 17.0
center diff: 7.0
center diff: 15.0
center diff: 24.0
center diff: 11.0
center diff: 33.0
center diff: 15.0
center diff: 15.0
center diff: 38.0
center diff: 75.0
center diff: 13.0
center diff: 0.0
center diff: 28.0
center diff: 10.0
center diff: 8.0
ce

In [69]:
# done

import subprocess
import os

pdf = "Ezcaray - Vozes"
# Define the paths and parameters
pdf_path = f"/home/sharvesh/Documents/Others/Human_AI/Test_sources/Test sources/{pdf}.pdf"
output_dir = f"/home/sharvesh/Documents/Others/Human_AI/Model/data/processed_book/{pdf}/pages"
trained_model = "/home/sharvesh/Documents/Others/Human_AI/Model/CRAFT/weights/craft_mlt_25k.pth"
refiner_model = "/home/sharvesh/Documents/Others/Human_AI/Model/CRAFT/weights/craft_refiner_CTW1500.pth"
text_threshold = 0.9
test_folder = output_dir  # Images folder created by process_main_utils.py
result_folder = f"/home/sharvesh/Documents/Others/Human_AI/Model/result/"
line_segments_folder = f"/home/sharvesh/Documents/Others/Human_AI/Model/data/{pdf}/line_segments"
padding = 50  #fixed(50)
min_width = 50
threshold = 0.7
margin = 0

# New parameters for process_main_utils.py
dpi = 300
remove_borders = True
noise_removal_area_threshold = 50
intensity_threshold = 128

# Ensure output directories exist
os.makedirs(output_dir, exist_ok=True)
os.makedirs(result_folder, exist_ok=True)
os.makedirs(line_segments_folder, exist_ok=True)

# Check if the output directory is empty
if not os.listdir(output_dir):
    print("Output directory is empty. Performing Step 1: Processing PDF into individual images.")
    # Step 1: Process the PDF into individual images
    subprocess.run([
        "python3", "process_main_utils.py",
        pdf_path, output_dir,
        "--dpi", str(dpi),
        "--remove_borders",
        "--noise_removal_area_threshold", str(noise_removal_area_threshold),
        "--intensity_threshold", str(intensity_threshold)
    ])
else:
    print("Output directory is not empty. Skipping Step 1.")

# Step 2: Run the text detection model on the processed images
print("Performing Step 2: Running the text detection model.")
subprocess.run([
    "python3", "/home/sharvesh/Documents/Others/Human_AI/Model/CRAFT/test.py",
    "--trained_model", trained_model,
    "--text_threshold", str(text_threshold),
    "--test_folder", output_dir,
    "--refine",
    "--refiner_model", refiner_model
])

# Step 3: Run the contour line splitter
print("Performing Step 3: Running the contour line splitter.")
subprocess.run([
    "python3", "contour_line_splitter.py",
    output_dir,  # Image directory from the first process
    result_folder,  # Contour directory from the second process
    line_segments_folder,  # Output directory for line segments
    "--padding", str(padding),
    "--min_width", str(min_width),
    "--threshold", str(threshold),
    "--margin", str(margin)
])

print("Processing completed successfully.")


Output directory is empty. Performing Step 1: Processing PDF into individual images.
Total pages in PDF: 11
Processed page 1/11
Processed page 2/11
Processed page 3/11
Processed page 4/11
Processed page 5/11
Processed page 6/11
Processed page 7/11
Processed page 8/11
Processed page 9/11
Processed page 10/11
Processed page 11/11
Processing completed!
Performing Step 2: Running the text detection model.


/home/sharvesh/.local/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/sharvesh/.local/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/home/sharvesh/Documents/Others/Human_AI/Model/CRAFT/test.py:128: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only`

Loading weights from checkpoint (/home/sharvesh/Documents/Others/Human_AI/Model/CRAFT/weights/craft_mlt_25k.pth)
Loading weights of refiner from checkpoint (/home/sharvesh/Documents/Others/Human_AI/Model/CRAFT/weights/craft_refiner_CTW1500.pth)
elapsed time : 52.58578896522522sDocuments/Others/Human_AI/Model/data/processed_book/Ezcaray - Vozes/pages/10.jpg
Performing Step 3: Running the contour line splitter.
in process dir func
Processing /home/sharvesh/Documents/Others/Human_AI/Model/data/processed_book/Ezcaray - Vozes/pages/7.jpg with contours /home/sharvesh/Documents/Others/Human_AI/Model/result/res_7.txt
HI
in get func
center diff: 2.0
center diff: 0.0
center diff: 46.0
center diff: 21.0
center diff: 9.0
center diff: 7.0
center diff: 54.0
center diff: 14.0
center diff: 5.0
center diff: 323.0
center diff: 66.0
center diff: 31.0
center diff: 32.0
center diff: 46.0
center diff: 32.0
center diff: 3.0
center diff: 2.0
center diff: 6.0
center diff: 53.0
center diff: 11.0
center diff: 9.

In [71]:
# done

import subprocess
import os

pdf = "Mendo - Principe perfecto"
# Define the paths and parameters
pdf_path = f"/home/sharvesh/Documents/Others/Human_AI/Test_sources/Test sources/{pdf}.pdf"
output_dir = f"/home/sharvesh/Documents/Others/Human_AI/Model/data/processed_book/{pdf}/pages"
trained_model = "/home/sharvesh/Documents/Others/Human_AI/Model/CRAFT/weights/craft_mlt_25k.pth"
refiner_model = "/home/sharvesh/Documents/Others/Human_AI/Model/CRAFT/weights/craft_refiner_CTW1500.pth"
text_threshold = 0.9
test_folder = output_dir  # Images folder created by process_main_utils.py
result_folder = f"/home/sharvesh/Documents/Others/Human_AI/Model/result/"
line_segments_folder = f"/home/sharvesh/Documents/Others/Human_AI/Model/data/{pdf}/line_segments"
padding = 50  #fixed(50)
min_width = 50
threshold = 0.7
margin = 0

# New parameters for process_main_utils.py
dpi = 300
remove_borders = True
noise_removal_area_threshold = 50
intensity_threshold = 128

# Ensure output directories exist
os.makedirs(output_dir, exist_ok=True)
os.makedirs(result_folder, exist_ok=True)
os.makedirs(line_segments_folder, exist_ok=True)

# Check if the output directory is empty
if not os.listdir(output_dir):
    print("Output directory is empty. Performing Step 1: Processing PDF into individual images.")
    # Step 1: Process the PDF into individual images
    subprocess.run([
        "python3", "process_main_utils.py",
        pdf_path, output_dir,
        "--dpi", str(dpi),
        "--remove_borders",
        "--noise_removal_area_threshold", str(noise_removal_area_threshold),
        "--intensity_threshold", str(intensity_threshold)
    ])
else:
    print("Output directory is not empty. Skipping Step 1.")

# Step 2: Run the text detection model on the processed images
print("Performing Step 2: Running the text detection model.")
subprocess.run([
    "python3", "/home/sharvesh/Documents/Others/Human_AI/Model/CRAFT/test.py",
    "--trained_model", trained_model,
    "--text_threshold", str(text_threshold),
    "--test_folder", output_dir,
    "--refine",
    "--refiner_model", refiner_model
])

# Step 3: Run the contour line splitter
print("Performing Step 3: Running the contour line splitter.")
subprocess.run([
    "python3", "contour_line_splitter.py",
    output_dir,  # Image directory from the first process
    result_folder,  # Contour directory from the second process
    line_segments_folder,  # Output directory for line segments
    "--padding", str(padding),
    "--min_width", str(min_width),
    "--threshold", str(threshold),
    "--margin", str(margin)
])

print("Processing completed successfully.")


Output directory is empty. Performing Step 1: Processing PDF into individual images.
Total pages in PDF: 9
Processed page 1/9
Processed page 2/9
Processed page 3/9
Processed page 4/9
Processed page 5/9
Processed page 6/9
Processed page 7/9
Processed page 8/9
Processed page 9/9
Processing completed!
Performing Step 2: Running the text detection model.


/home/sharvesh/.local/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/sharvesh/.local/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/home/sharvesh/Documents/Others/Human_AI/Model/CRAFT/test.py:128: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only`

Loading weights from checkpoint (/home/sharvesh/Documents/Others/Human_AI/Model/CRAFT/weights/craft_mlt_25k.pth)
Loading weights of refiner from checkpoint (/home/sharvesh/Documents/Others/Human_AI/Model/CRAFT/weights/craft_refiner_CTW1500.pth)
elapsed time : 40.160757541656494suments/Others/Human_AI/Model/data/processed_book/Mendo - Principe perfecto/pages/8.jpg
Performing Step 3: Running the contour line splitter.
in process dir func
Processing /home/sharvesh/Documents/Others/Human_AI/Model/data/processed_book/Mendo - Principe perfecto/pages/7.jpg with contours /home/sharvesh/Documents/Others/Human_AI/Model/result/res_7.txt
HI
in get func
center diff: 2.0
center diff: 6.0
center diff: 72.0
center diff: 129.0
center diff: 118.0
center diff: 2.0
center diff: 37.0
center diff: 21.0
center diff: 0.0
center diff: 0.0
center diff: 25.0
center diff: 22.0
center diff: 57.0
center diff: 18.0
center diff: 14.0
center diff: 32.0
center diff: 23.0
center diff: 3.0
center diff: 1.0
center diff: 0

In [73]:
#done

import subprocess
import os

pdf = "Paredes - Reglas generales"
# Define the paths and parameters
pdf_path = f"/home/sharvesh/Documents/Others/Human_AI/Test_sources/Test sources/{pdf}.pdf"
output_dir = f"/home/sharvesh/Documents/Others/Human_AI/Model/data/processed_book/{pdf}/pages"
trained_model = "/home/sharvesh/Documents/Others/Human_AI/Model/CRAFT/weights/craft_mlt_25k.pth"
refiner_model = "/home/sharvesh/Documents/Others/Human_AI/Model/CRAFT/weights/craft_refiner_CTW1500.pth"
text_threshold = 0.9
test_folder = output_dir  # Images folder created by process_main_utils.py
result_folder = f"/home/sharvesh/Documents/Others/Human_AI/Model/result/"
line_segments_folder = f"/home/sharvesh/Documents/Others/Human_AI/Model/data/{pdf}/line_segments"
padding = 50  #fixed(50)
min_width = 50
threshold = 0.7
margin = 0

# New parameters for process_main_utils.py
dpi = 300
remove_borders = True
noise_removal_area_threshold = 50
intensity_threshold = 128

# Ensure output directories exist
os.makedirs(output_dir, exist_ok=True)
os.makedirs(result_folder, exist_ok=True)
os.makedirs(line_segments_folder, exist_ok=True)

# Check if the output directory is empty
if not os.listdir(output_dir):
    print("Output directory is empty. Performing Step 1: Processing PDF into individual images.")
    # Step 1: Process the PDF into individual images
    subprocess.run([
        "python3", "process_main_utils.py",
        pdf_path, output_dir,
        "--dpi", str(dpi),
        "--remove_borders",
        "--noise_removal_area_threshold", str(noise_removal_area_threshold),
        "--intensity_threshold", str(intensity_threshold)
    ])
else:
    print("Output directory is not empty. Skipping Step 1.")

# Step 2: Run the text detection model on the processed images
print("Performing Step 2: Running the text detection model.")
subprocess.run([
    "python3", "/home/sharvesh/Documents/Others/Human_AI/Model/CRAFT/test.py",
    "--trained_model", trained_model,
    "--text_threshold", str(text_threshold),
    "--test_folder", output_dir,
    "--refine",
    "--refiner_model", refiner_model
])

# Step 3: Run the contour line splitter
print("Performing Step 3: Running the contour line splitter.")
subprocess.run([
    "python3", "contour_line_splitter.py",
    output_dir,  # Image directory from the first process
    result_folder,  # Contour directory from the second process
    line_segments_folder,  # Output directory for line segments
    "--padding", str(padding),
    "--min_width", str(min_width),
    "--threshold", str(threshold),
    "--margin", str(margin)
])

print("Processing completed successfully.")


Output directory is empty. Performing Step 1: Processing PDF into individual images.
Total pages in PDF: 9
Processed page 1/9
Processed page 2/9
Processed page 3/9
Processed page 4/9
Processed page 5/9
Processed page 6/9
Processed page 7/9
Processed page 8/9
Processed page 9/9
Processing completed!
Performing Step 2: Running the text detection model.


/home/sharvesh/.local/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/sharvesh/.local/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/home/sharvesh/Documents/Others/Human_AI/Model/CRAFT/test.py:128: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only`

Loading weights from checkpoint (/home/sharvesh/Documents/Others/Human_AI/Model/CRAFT/weights/craft_mlt_25k.pth)
Loading weights of refiner from checkpoint (/home/sharvesh/Documents/Others/Human_AI/Model/CRAFT/weights/craft_refiner_CTW1500.pth)
elapsed time : 45.67190623283386scuments/Others/Human_AI/Model/data/processed_book/Paredes - Reglas generales/pages/8.jpg
Performing Step 3: Running the contour line splitter.
in process dir func
Processing /home/sharvesh/Documents/Others/Human_AI/Model/data/processed_book/Paredes - Reglas generales/pages/7.jpg with contours /home/sharvesh/Documents/Others/Human_AI/Model/result/res_7.txt
HI
in get func
center diff: 76.0
center diff for skipped: 1077.0
center diff: 360.0
center diff: 98.0
center diff: 28.0
center diff: 48.0
center diff: 69.0
center diff: 119.0
center diff: 632.0
center diff: 104.0
center diff: 21.0
center diff: 34.0
center diff: 57.0
center diff: 14.0
center diff: 146.0
center diff: 126.0
center diff: 84.0
center diff: 28.0
cente

In [ ]:
#done 

import subprocess
import os

pdf = "PORCONES.228.35 – 1636"
# Define the paths and parameters
pdf_path = f"/home/sharvesh/Documents/Others/Human_AI/Test_sources/Test sources/{pdf}.pdf"
output_dir = f"/home/sharvesh/Documents/Others/Human_AI/Model/data/processed_book/{pdf}/pages"
trained_model = "/home/sharvesh/Documents/Others/Human_AI/Model/CRAFT/weights/craft_mlt_25k.pth"
refiner_model = "/home/sharvesh/Documents/Others/Human_AI/Model/CRAFT/weights/craft_refiner_CTW1500.pth"
text_threshold = 0.9
test_folder = output_dir  # Images folder created by process_main_utils.py
result_folder = f"/home/sharvesh/Documents/Others/Human_AI/Model/result/"
line_segments_folder = f"/home/sharvesh/Documents/Others/Human_AI/Model/data/{pdf}/line_segments"
padding = 50  #fixed(50)
min_width = 50
threshold = 0.7
margin = 0

# New parameters for process_main_utils.py
dpi = 300
remove_borders = True
noise_removal_area_threshold = 50
intensity_threshold = 128

# Ensure output directories exist
os.makedirs(output_dir, exist_ok=True)
os.makedirs(result_folder, exist_ok=True)
os.makedirs(line_segments_folder, exist_ok=True)

# Check if the output directory is empty
if not os.listdir(output_dir):
    print("Output directory is empty. Performing Step 1: Processing PDF into individual images.")
    # Step 1: Process the PDF into individual images
    subprocess.run([
        "python3", "process_main_utils.py",
        pdf_path, output_dir,
        "--dpi", str(dpi),
        "--remove_borders",
        "--noise_removal_area_threshold", str(noise_removal_area_threshold),
        "--intensity_threshold", str(intensity_threshold)
    ])
else:
    print("Output directory is not empty. Skipping Step 1.")

# Step 2: Run the text detection model on the processed images
print("Performing Step 2: Running the text detection model.")
subprocess.run([
    "python3", "/home/sharvesh/Documents/Others/Human_AI/Model/CRAFT/test.py",
    "--trained_model", trained_model,
    "--text_threshold", str(text_threshold),
    "--test_folder", output_dir,
    "--refine",
    "--refiner_model", refiner_model
])

# Step 3: Run the contour line splitter
print("Performing Step 3: Running the contour line splitter.")
subprocess.run([
    "python3", "contour_line_splitter.py",
    output_dir,  # Image directory from the first process
    result_folder,  # Contour directory from the second process
    line_segments_folder,  # Output directory for line segments
    "--padding", str(padding),
    "--min_width", str(min_width),
    "--threshold", str(threshold),
    "--margin", str(margin)
])

print("Processing completed successfully.")


In [3]:
pip install python-docx

Note: you may need to restart the kernel to use updated packages.


In [19]:
import os
from pathlib import Path
import re
from docx import Document

# Define project directory
project_dir = "/home/sharvesh/Documents/Others/Human_AI/Model/"
docx_dir = "/home/sharvesh/Documents/Others/Human_AI/Test_transcriptions/Test transcriptions"
ground_truth_dir = os.path.join(project_dir, "ground_truth")

# Create ground_truth directory
Path(ground_truth_dir).mkdir(parents=True, exist_ok=True)

# Define sources (based on your project)
sources = [
    "Paredes transcription"
]

# Function to preprocess text according to transcription rules
def preprocess_text(text):
    # Replace v with u
    text = text.replace('v', 'u').replace('V', 'U')
    # Replace ç with z
    text = text.replace('ç', 'z').replace('Ç', 'Z')
    # Remove accents except ñ/Ñ
    accents = {'á': 'a', 'é': 'e', 'í': 'i', 'ó': 'o', 'ú': 'u',
               'Á': 'A', 'É': 'E', 'Í': 'I', 'Ó': 'O', 'Ú': 'U'}
    for accented, unaccented in accents.items():
        text = text.replace(accented, unaccented)
    # Expand macrons (simplified: assume ¯ after a vowel or q)
    # a¯ → an, e¯ → en, etc.; q¯ → que
    text = re.sub(r'([aeiouAEIOU])¯', r'\1n', text)  # e.g., a¯ → an
    text = re.sub(r'q¯', 'que', text)  # e.g., q¯ → que
    # Replace long s (ſ) with s
    text = text.replace('ſ', 's')
    return text

# Function to process a .docx file and save preprocessed transcriptions
def process_docx(docx_path, source_name):
    # Create source directory in ground_truth
    source_dir = os.path.join(ground_truth_dir, source_name)
    Path(source_dir).mkdir(parents=True, exist_ok=True)

    # Read the .docx file
    doc = Document(docx_path)
    current_page = None
    page_lines = []

    for para in doc.paragraphs:
        text = para.text.strip()
        if not text:
            continue

        # Check for page markers (e.g., "PDF p1", "PDF p2", "PDF p3")
        if text.startswith("PDF p"):
            # Save previous page if exists
            if current_page is not None and page_lines:
                output_path = os.path.join(source_dir, f"page{current_page}.txt")
                with open(output_path, "w", encoding="utf-8") as f:
                    f.write("\n".join(page_lines))
                print(f"Saved ground truth: {output_path}")
                page_lines = []

            # Extract page number
            page_num = text.replace("PDF p", "")
            try:
                current_page = int(page_num)
            except ValueError:
                print(f"Invalid page number in {source_name}: {text}")
                continue

        elif text.startswith("NOTES:"):
            # Skip notes section
            continue

        else:
            # Preprocess the line
            preprocessed_line = preprocess_text(text)
            if preprocessed_line:
                page_lines.append(preprocessed_line)

    # Save the last page
    if current_page is not None and page_lines:
        output_path = os.path.join(source_dir, f"page{current_page}.txt")
        with open(output_path, "w", encoding="utf-8") as f:
            f.write("\n".join(page_lines))
        print(f"Saved ground truth: {output_path}")

# Process each source's .docx file
for source in sources:
    docx_filename = f"{source}.docx"
    docx_path = os.path.join(docx_dir, docx_filename)
    if not os.path.exists(docx_path):
        print(f"Docx file not found for {source}: {docx_path}")
        continue
    print(f"Processing {docx_filename}...")
    process_docx(docx_path, source)

Processing Paredes transcription.docx...
HI
HI
Saved ground truth: /home/sharvesh/Documents/Others/Human_AI/Model/ground_truth/Paredes transcription/page1.txt
HI
Saved ground truth: /home/sharvesh/Documents/Others/Human_AI/Model/ground_truth/Paredes transcription/page2.txt
Saved ground truth: /home/sharvesh/Documents/Others/Human_AI/Model/ground_truth/Paredes transcription/page3.txt


In [3]:
import os
import glob
import pandas as pd

# Define the list of book names and their corresponding number of pages
book_info = {
    "Buendia - Instruccion": 5,
    "Constituciones sinodales Calahorra 1602": 3,
    "Ezcaray - Vozes": 3,
    "Mendo - Principe perfecto": 5,
    "Paredes - Reglas generales": 3,
    "PORCONES.228.35 - 1636": 3
}

# Base paths for line segments and ground truth
base_line_segments_path = "/home/sharvesh/Documents/Others/Human_AI/Model/data"
base_ground_truth_path = "/home/sharvesh/Documents/Others/Human_AI/Model/ground_truth"
output_csv = "dataset.csv"

# Ensure the base ground truth path exists
if not os.path.exists(base_ground_truth_path):
    raise FileNotFoundError(f"Base ground truth path {base_ground_truth_path} does not exist. Please provide the correct path.")

# Initialize a list to store the dataset
dataset = []

# Iterate through each book
for pdf_name, num_pages in book_info.items():
    print(f"\nProcessing book: {pdf_name} (Expected {num_pages} pages)")

    # Define paths for this book
    line_segments_folder = os.path.join(base_line_segments_path, pdf_name, "line_segments")
    ground_truth_folder = os.path.join(base_ground_truth_path, pdf_name)

    # Ensure the line segments folder exists
    if not os.path.exists(line_segments_folder):
        print(f"Warning: Line segments folder {line_segments_folder} not found. Skipping book {pdf_name}.")
        continue

    # Ensure the ground truth folder exists
    if not os.path.exists(ground_truth_folder):
        print(f"Warning: Ground truth folder {ground_truth_folder} not found. Skipping book {pdf_name}.")
        continue

    # Get all page directories in the line segments folder for this book
    page_dirs = sorted(glob.glob(os.path.join(line_segments_folder, "*")), key=lambda x: int(os.path.basename(x)))

    # Filter page directories to only include the expected number of pages
    page_dirs = [page_dir for page_dir in page_dirs if int(os.path.basename(page_dir)) < num_pages]

    # Check if the number of page directories matches the expected number of pages
    if len(page_dirs) != num_pages:
        print(f"Warning: Expected {num_pages} pages for book {pdf_name}, but found {len(page_dirs)} page directories.")

    # Iterate through each page directory
    for page_dir in page_dirs:
        page_index = int(os.path.basename(page_dir))  # e.g., 0 for page 0 (convert to int)
        print(f"Processing page {page_index} in book {pdf_name}...")

        # Load the ground truth file for this page
        # Ground truth files are named as 1.txt, 2.txt, ..., so we use page_index + 1
        gt_file = os.path.join(ground_truth_folder, f"page{page_index + 1}.txt")
        if not os.path.exists(gt_file):
            print(f"Warning: Ground truth file {gt_file} not found. Skipping page {page_index} in book {pdf_name}.")
            continue

        # Read the ground truth lines
        with open(gt_file, 'r', encoding='utf-8') as f:
            gt_lines = [line.strip() for line in f.readlines() if line.strip()]
        
        # Get all line segment images for this page
        line_images = sorted(glob.glob(os.path.join(page_dir, "*.jpg")), key=lambda x: int(os.path.basename(x).split('.')[0]))
        
        # Check if the number of line segment images matches the number of ground truth lines
        if len(line_images) != len(gt_lines):
            print(f"Warning: Mismatch in page {page_index} of book {pdf_name}. Found {len(line_images)} line segment images but {len(gt_lines)} ground truth lines.")
            # Adjust the number of lines to match the smaller of the two
            min_length = min(len(line_images), len(gt_lines))
            line_images = line_images[:min_length]
            gt_lines = gt_lines[:min_length]
        
        # Pair each line segment image with its ground truth
        for line_image, gt_line in zip(line_images, gt_lines):
            dataset.append((line_image, gt_line))
            print(f"Paired {line_image} with ground truth: {gt_line}")

# Create a DataFrame from the dataset
df = pd.DataFrame(dataset, columns=["image_path", "text"])

# Save the DataFrame to a CSV file
output_csv = "/home/sharvesh/Documents/Others/Human_AI/Model/data/output/dataset.csv"  # Update this to your desired path

# Ensure the output directory exists
os.makedirs(os.path.dirname(output_csv), exist_ok=True)

# Save the DataFrame to the specified path
df.to_csv(output_csv, index=False, encoding='utf-8-sig')
print(f"\nSaved dataset to {output_csv}")


Processing book: Buendia - Instruccion (Expected 5 pages)
Processing page 0 in book Buendia - Instruccion...
Paired /home/sharvesh/Documents/Others/Human_AI/Model/data/Buendia - Instruccion/line_segments/0/1.jpg with ground truth: Al
Paired /home/sharvesh/Documents/Others/Human_AI/Model/data/Buendia - Instruccion/line_segments/0/2.jpg with ground truth: INFINITAMENTE AMABLE
Paired /home/sharvesh/Documents/Others/Human_AI/Model/data/Buendia - Instruccion/line_segments/0/3.jpg with ground truth: NIÑO JESUS.
Paired /home/sharvesh/Documents/Others/Human_AI/Model/data/Buendia - Instruccion/line_segments/0/4.jpg with ground truth: A Uos, Dulcissimo Niño
Paired /home/sharvesh/Documents/Others/Human_AI/Model/data/Buendia - Instruccion/line_segments/0/5.jpg with ground truth: JESUS, que no solo os
Paired /home/sharvesh/Documents/Others/Human_AI/Model/data/Buendia - Instruccion/line_segments/0/6.jpg with ground truth: dignasteis de llamaros
Paired /home/sharvesh/Documents/Others/Human_AI/Model/